<a href="https://colab.research.google.com/github/LeonardoMafra/Ia-Generativa/blob/main/chatLangchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import yaml
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate
from langchain_community.document_loaders import TextLoader

In [ ]:
with open('config.yaml','r') as config_file:
    config = yaml.safe_load(config_file)

In [ ]:
gemini = ChatGoogleGenerativeAI(
    model = "gemini-2.0-flash",
    google_api_key=config['KEY'],
    temperature=0.3
)

In [ ]:
def load_knowledge_base(file_path):
    """Carrega a base de conhecimento de um arquivo de texto"""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        # Retorna conteúdo de exemplo se arquivo não existir
        return """PROBLEMAS COMUNS COM CARREGADORES DE CELULAR:
CARREGAMENTO NÃO INICIA / CELULAR NÃO CARREGA:

Verifique se a tomada está funcionando corretamente.

Confirme se o plugue do carregador está bem encaixado.

Teste o cabo USB com outro adaptador ou computador.

Verifique se o cabo USB está danificado (desfiado, com conectores tortos).

Inspecione a porta de carregamento do celular para sujeira ou obstruções.

SOLUÇÕES AVANÇADAS:

Teste o carregador com outro dispositivo para garantir que o problema não é no celular.

Se o celular não carregar com o carregador original, tente com um adaptador novo.

Reinicie o celular e conecte-o ao carregador novamente.

Se o celular estiver com a bateria completamente descarregada, aguarde alguns minutos antes de ver o ícone de carregamento.

MANUTENÇÃO PREVENTIVA:

Realize limpezas periódicas na porta de carregamento do celular (mensalmente, ou conforme necessário).

Não deixe o carregador ou cabo expostos a locais úmidos ou danificados.

Evite forçar o cabo ou adaptador ao conectar no celular.

Use cabos e adaptadores originais ou de boa qualidade."""

In [ ]:
# carregar base de conhecimento

knowledge_base_content = load_knowledge_base('manual_carregador.txt')


# historico de conversas simulado

historico_conversa = '''Cliente: Por que meu celular não está carregando mesmo estando na tomada?.
Chatbot : Você ja verificou se a tomada esta funcionando corretamente e se o plug do carregador esta bem encaixado?
Cliente : Sim, já verifiquei e não há nada errado'''

# nova pergunta do cliente

pergunta = '''O carregador continua conectado ao celular e a tomada, porem o celular nao da sinais de estar caregando'''

inputs={
    "context" : knowledge_base_content,
    "question": pergunta,
    "historico": historico_conversa
}


In [ ]:
#template para consulta na base de conhecimento

prompt_base_conhecimento = PromptTemplate(
    input_variables=["context","question"],
    template = """Use o seguinte contexto tecnico para responder a pergunta sobre carregadores de celular.
    Responda apenas com base nas informações fornecidas na base de conhecimento
    Seja especifico e tecnico, mas mantenha a linguagem acessivel.
    Não sugira procedimentos que não estejam documentados no contexto.
    Base de Conhecimento: {context}
    Pergunta do Cliete: {question}
    Resposta Tecnica baseada na base de conhecimento: """
)


In [ ]:
# Template para análise do histórico
prompt_historico_conversas = PromptTemplate(
    input_variables=["historico", "question"],
    template="""Analise o histórico da conversa para entender o contexto do problema.
    Use essas informações para complementar sua resposta, evitando repetir soluções já tentadas.
    Considere a evolução do problema descrito pelo cliente.

    Histórico da Conversa: {historico}

    Pergunta Atual: {question}

    Análise contextual do histórico:"""
)


# Template para resposta final combinada
prompt_final = PromptTemplate(
    input_variables=["resposta_base_conhecimento", "resposta_historico_conversas"],
    template="""Combine as informações técnicas e o contexto histórico para criar uma resposta completa.
    Priorize soluções que ainda não foram tentadas pelo cliente.
    Se necessário, sugira escalonamento para técnico especializado.
    Mantenha tom profissional e prestativo.

    Informações Técnicas: {resposta_base_conhecimento}

    Contexto da Conversa: {resposta_historico_conversas}

    Resposta Final Integrada:"""
)

In [ ]:
print("== CONFIGURAÇÃO CONCLUIDA ==")
print(f"Base de conhecimento carregada: {len(knowledge_base_content)} caracteres")
print("Pergunta do cliente: {pergunta}")

chain_base_conhecimento = prompt_base_conhecimento | gemini
chain_historico_conversas = prompt_historico_conversas | gemini
chain_final = prompt_final | gemini

# executar primeiro chain da base de conhecimento

print("1. Consultando base de conhecimento...")
resultado_base_conhecimento = chain_base_conhecimento.invoke({
    "context" : inputs["context"],
    "question" : inputs["question"]
})

print("2. Analisando historico da conversa...")
resultado_historico_conversas = chain_historico_conversas.invoke({
    "historico" : inputs["historico"],
    "question" : inputs["question"]
})

# executar chain final - combinação

print("3. Gerando resposta integrada...")
resultado_final = chain_final.invoke({
    "resposta_base_conhecimento" : resultado_base_conhecimento.content,
    "resposta_historico_conversas" : resultado_historico_conversas.content
})


print("\n ANALISE DA BASE DE CONHECIMENTO:")
print("-" * 40)
print(resultado_base_conhecimento.content)

== CONFIGURAÇÃO CONCLUIDA ==
Base de conhecimento carregada: 1095 caracteres
Pergunta do cliente: {pergunta}
1. Consultando base de conhecimento...
2. Analisando historico da conversa...
3. Gerando resposta integrada...

 ANALISE DA BASE DE CONHECIMENTO:
----------------------------------------
O celular não está carregando? Aqui estão algumas verificações que você pode fazer:

1.  **Tomada:** Veja se a tomada está funcionando.
2.  **Conexão:** Confirme se o carregador está bem encaixado na tomada e no celular.
3.  **Cabo USB:**
    *   Teste o cabo USB em outro adaptador ou computador.
    *   Verifique se o cabo USB está danificado (desfiado, conectores tortos).
4.  **Porta de Carregamento:** Inspecione a porta de carregamento do celular para ver se tem sujeira ou algo bloqueando.

Se o problema persistir:

1.  **Teste com outro dispositivo:** Teste o carregador com outro celular para ter certeza de que o problema não é no seu celular.
2.  **Adaptador novo:** Se não carregar com o ca

In [ ]:
print("\n Resposta final para o cliente:")
print("-" * 40)
print(resultado_final.content)


 Resposta final para o cliente:
----------------------------------------
Prezado(a) cliente,

Entendo que você já verificou a tomada e a conexão do carregador, e o problema persiste. Para solucionar essa questão, vamos explorar algumas alternativas que ainda não foram mencionadas:

1.  **Teste com outro cabo USB:** Mesmo que o cabo pareça visualmente em bom estado, pode haver um dano interno. Utilize um cabo USB diferente, preferencialmente um que você saiba que está funcionando corretamente, para verificar se o problema está no cabo original.

2.  **Limpeza da porta de carregamento:** Utilize uma fonte de ar comprimido (como aquelas usadas para limpar teclados) para remover qualquer sujeira ou resíduo que possa estar obstruindo a porta de carregamento do seu celular. Se não tiver ar comprimido, um palito de dente (desligue o aparelho antes) pode ser usado com extrema cautela para remover qualquer sujeira visível. **Importante:** Seja muito cuidadoso para não danificar os pinos intern